In [1]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt

In [2]:
DATA_URL = "https://raw.githubusercontent.com/PabloJRW/used-car-price-prediction/main/datasets/raw/used_cars.csv"
df = pd.read_csv(DATA_URL, delimiter=',')
df.head(10)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"
5,Acura,ILX 2.4L,2016,"136,397 mi.",Gasoline,2.4 Liter,F,Silver,Ebony.,None reported,NaN,"$14,798"
6,Audi,S3 2.0T Premium Plus,2017,"84,000 mi.",Gasoline,292.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,6-Speed A/T,Blue,Black,None reported,Yes,"$31,000"
7,BMW,740 iL,2001,"242,000 mi.",Gasoline,282.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,Green,Green,None reported,Yes,"$7,300"
8,Lexus,RC 350 F Sport,2021,"23,436 mi.",Gasoline,311.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,6-Speed A/T,Black,Black,None reported,Yes,"$41,927"
9,Tesla,Model X Long Range Plus,2020,"34,000 mi.",NaN,534.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,"$69,950"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


In [4]:
# Verificanco los valores de 'milage'
df['milage']

0       51,000 mi.
1       34,742 mi.
2       22,372 mi.
3       88,900 mi.
4        9,835 mi.
           ...    
4004       714 mi.
4005    10,900 mi.
4006     2,116 mi.
4007    33,000 mi.
4008    43,000 mi.
Name: milage, Length: 4009, dtype: object

In [5]:
# Se encuentra 'mi.' en todas las entradas de 'mi.'
df['milage'].str.split().str[1].unique()

array(['mi.'], dtype=object)

In [6]:
# Extrayendo sólo los numeros, excluyendo 'mi'
df['milage'] = df['milage'].str.split().str[0]
# Convirtiendo la variable 'milage' a tipo numérico
df['milage'] = df['milage'].str.replace(",", "").astype(int)

# Eliminando caracteres y convirtiendo la variable 'price' a numérica
df['price'] = df['price'].str.replace("$", "").str.replace(",", "").astype(int)

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
model_year,4009.0,2015.515590,6.104816,1974.0,2012.0,2017.0,2020.0,2024.0
milage,4009.0,64717.551010,52296.599459,100.0,23044.0,52775.0,94100.0,405000.0
price,4009.0,44553.190322,78710.635554,2000.0,17200.0,31000.0,49990.0,2954083.0


In [8]:
# ¿Cuántos valores únicos tiene cada variable categórica?
for col in df.select_dtypes('object'):
    print(f"{col.ljust(15)}: {df[col].nunique()}")

brand          : 57
model          : 1898
fuel_type      : 7
engine         : 1146
transmission   : 62
ext_col        : 319
int_col        : 156
accident       : 2
clean_title    : 1


In [9]:
# Cuántos tipos de combustible se encuentran en el dataset?
[print(x) for x in df['fuel_type'].unique()]

E85 Flex Fuel
Gasoline
Hybrid
nan
Diesel
Plug-In Hybrid
–
not supported


[None, None, None, None, None, None, None, None]

In [10]:
# Características de los autos que funcionan con combustible 'E85 Flex Fuel'
df[df['fuel_type']=='E85 Flex Fuel'][['brand','model','model_year','milage','engine','price']].sample(10)

,brand,model,model_year,milage,engine,price
3420,Land,Rover Range Rover Sport Supercharged,2015,64300,550.0HP 5.0L 8 Cylinder Engine Flex Fuel Capab...,44850
3019,Chrysler,200 C,2015,170000,295.0HP 3.6L V6 Cylinder Engine Flex Fuel Capa...,5995
3844,Ford,F-250 XL,2020,36000,385.0HP 6.2L 8 Cylinder Engine Flex Fuel Capab...,44000
1725,Chevrolet,Tahoe LTZ,2014,196300,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,10995
154,Ford,E150 XLT,2013,162100,255.0HP 5.4L 8 Cylinder Engine Flex Fuel Capab...,12500
2921,Mercedes-Benz,M-Class ML 350 4MATIC,2015,88000,302.0HP 3.5L V6 Cylinder Engine Flex Fuel Capa...,19500
3566,Chevrolet,Suburban LT,2016,98000,355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,30000
2465,GMC,Savana 2500 Work Van,2017,117000,342.0HP 6.0L 8 Cylinder Engine Flex Fuel Capab...,38000
2721,Ford,Explorer XLT,2017,53000,290.0HP 3.5L V6 Cylinder Engine Flex Fuel Capa...,25000
1990,Chevrolet,Silverado 1500 1LZ,2014,111095,355.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,26000


In [12]:
# Características de los autos que funcionan con combustible 'Gasoline'
df[df['fuel_type']=='Gasoline'][['brand','model','model_year','milage','engine','price']].sample(10)

,brand,model,model_year,milage,engine,price
3882,Mazda,MX-5 Miata Base,1999,130000,140.0HP 1.8L 4 Cylinder Engine Gasoline Fuel,9000
751,BMW,M4 Base,2018,19600,425.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,62000
2780,BMW,Z4 sDrive28i,2014,15889,240.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,34500
1825,Alfa,Romeo Stelvio Base,2018,50000,280.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,27500
3645,BMW,750 i xDrive,2014,85095,445.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,24900
287,Porsche,Cayman Base,2014,52000,275.0HP 2.7L Flat 6 Cylinder Engine Gasoline Fuel,37000
2076,MINI,Cooper Base,2003,103726,115.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,5995
1258,Bentley,Continental GTC Base,2013,79422,616.0HP 6.0L 12 Cylinder Engine Gasoline Fuel,64000
2957,Lamborghini,Huracan LP580-2,2017,14586,"5.2L V-10 gasoline direct injection, DOHC, var...",209991
1549,INFINITI,Q50 3.0t Red Sport 400,2020,23344,3.0L V6 24V GDI DOHC Twin Turbo,36999


In [13]:
df[df['fuel_type']=='Gasoline'][['engine']].sample(10)

,engine
1040,335.0HP 3.6L V6 Cylinder Engine Gasoline Fuel
371,2.5L I4 16V GDI DOHC
2983,181.0HP 4.0L Straight 6 Cylinder Engine Gasoli...
4008,248.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
265,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel
2586,235.0HP 3.2L Straight 6 Cylinder Engine Gasoli...
3917,170.0HP 2.5L 5 Cylinder Engine Gasoline Fuel
3218,552.0HP 6.0L 12 Cylinder Engine Gasoline Fuel
2011,300.0HP 3.7L V6 Cylinder Engine Gasoline Fuel
1547,416.0HP 4.0L 8 Cylinder Engine Gasoline Fuel


In [14]:
df[df['fuel_type']=='not supported']

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
2894,Toyota,Mirai Limited,2021,44000,not supported,182.0HP Electric Motor Hydrogen Fuel,A/T,Gray,Black,None reported,Yes,18500
3700,Toyota,Mirai Base,2016,40000,not supported,151.0HP Electric Motor Hydrogen Fuel,A/T,Silver,Black,None reported,Yes,9500


In [15]:
df[df['fuel_type'].isna()][:10]

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
9,Tesla,Model X Long Range Plus,2020,34000,NaN,534.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,69950
44,Lucid,Air Grand Touring,2022,3552,NaN,536.0HP Electric Motor Electric Fuel System,1-Speed A/T,Red,Beige,None reported,Yes,119999
68,Lucid,Air Grand Touring,2022,4900,NaN,536.0HP Electric Motor Electric Fuel System,1-Speed A/T,Red,Black,None reported,Yes,99000
92,Rivian,R1S Adventure Package,2023,2800,NaN,835.0HP Electric Motor Electric Fuel System,1-Speed A/T,White,Green,None reported,Yes,92000
122,Rivian,R1S Adventure Package,2023,2500,NaN,835.0HP Electric Motor Electric Fuel System,A/T,Green,White,None reported,Yes,94000
129,Lucid,Air Pure,2023,1300,NaN,620.0HP Electric Motor Electric Fuel System,A/T,Black,Gray,NaN,NaN,86900
155,Tesla,Model 3 Performance,2022,13079,NaN,455.0HP Electric Motor Electric Fuel System,A/T,Black,White,None reported,Yes,47000
189,Tesla,Model Y Performance,2023,500,NaN,455.0HP Electric Motor Electric Fuel System,1-Speed A/T,Black,White,None reported,Yes,60000
225,Tesla,Model X Base,2023,8200,NaN,670.0HP Electric Motor Electric Fuel System,A/T,Black,Black,None reported,Yes,93999
236,Polestar,2 Launch Edition,2021,12172,NaN,Electric,1-Speed Automatic,Thunder Gray,Charcoal,None reported,NaN,35999


In [16]:
df['fuel_type'] = df['fuel_type'].replace(np.nan, 'electric')
df['fuel_type'] = df['fuel_type'].replace('not supported', 'hydrogen')

In [17]:
df['fuel_type'].unique()

array(['E85 Flex Fuel', 'Gasoline', 'Hybrid', 'electric', 'Diesel',
       'Plug-In Hybrid', '–', 'hydrogen'], dtype=object)

In [31]:
import re

def extraer_caracteristicas_engine(engine):
    # Expresión regular para extraer la potencia 'hp'
    hp_pattern = r'(\d+\.\d+)HP'
    hp_matches = re.search(hp_pattern, engine)

    # Expresión regular para extraer la cilindrada 'L'
    cylinders_pattern = r'(\d+\.\d+)L'
    cylinders_matches = re.search(cylinders_pattern, engine)

    # Expresión regular para extraer el tipo de motor
    motor_pattern = r'(\w+\s?\w+\s?\w*) Engine'
    motor_matches = re.search(motor_pattern, engine)

    # Expresión regular para extraer el combustible
    fuel_pattern = r'(\w+\s?\w+\s?\w*) Fuel'
    fuel_matches = re.search(fuel_pattern, engine)

    # Asignar valores predeterminados en caso de no encontrar coincidencias
    hp = hp_matches.group(1) if hp_matches else None
    cylinders = cylinders_matches.group(1) if cylinders_matches else None
    motor = motor_matches.group(1) if motor_matches else None
    fuel = fuel_matches.group(1) if fuel_matches else None

    return hp, cylinders, motor, fuel


In [29]:
def extraer_caracteristicas_engine(engine):
    # Expresión regular para extraer la potencia 'hp'
    hp_pattern = r'(\d+\.\d+)HP'
    hp_matches = re.search(hp_pattern, engine)

    # Expresión regular para extraer la cilindrada 'cylinders' 
    cylinders_pattern = r'(\d+\.\d+)L'
    cylinders_matches = re.search(cylinders_pattern, engine)

    # Expresión regular para extraer la potencia 'hp'
    motor_pattern = r'(\w+\s?\w+\s?\w*)'
    motor_matches = re.search(motor_pattern, engine)

    # Expresión regular para extraer la cilindrada 'cylinders' 
    fuel_pattern = r'(\d+\.\d+)'
    fuel_matches = re.search(fuel_pattern, engine)

    # Asignar valores predeterminados en caso de no encontrar coincidencias
    return hp_matches.group(0), cylinders_matches, motor_matches, fuel_matches

In [33]:
df['hp'], df['cylinders'], df['motor_matches'], df['fuel_matches'] = zip(*df['engine'].apply(extraer_caracteristicas_engine))

In [34]:

df

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,hp,cylinders,motor_matches,fuel_matches
0,Ford,Utility Police Interceptor Base,2013,51000,E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,10300,300.0,3.7,7L V6 Cylinder,Cylinder Engine Flex
1,Hyundai,Palisade SEL,2021,34742,Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,38005,None,3.8,None,None
2,Lexus,RX 350 RX 350,2022,22372,Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,54598,None,None,None,None
3,INFINITI,Q50 Hybrid Sport,2015,88900,Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,15500,354.0,3.5,5L V6 Cylinder,None
4,Audi,Q3 45 S line Premium Plus,2021,9835,Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,34999,None,2.0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,Bentley,Continental GT Speed,2023,714,Gasoline,6.0L W12 48V PDI DOHC Twin Turbo,8-Speed Automatic with Auto-Shift,C / C,Hotspur,None reported,Yes,349950,None,6.0,None,None
4005,Audi,S4 3.0T Premium Plus,2022,10900,Gasoline,349.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,53900,349.0,3.0,0L V6 Cylinder,Cylinder Engine Gasoline
4006,Porsche,Taycan,2022,2116,electric,Electric,Automatic,Black,Black,None reported,NaN,90998,None,None,None,None
4007,Ford,F-150 Raptor,2020,33000,Gasoline,450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,Blue,Black,None reported,Yes,62999,450.0,3.5,5L V6 Cylinder,Cylinder Engine Gasoline


In [35]:
df['engine']

0       300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...
1                                    3.8L V6 24V GDI DOHC
2                                          3.5 Liter DOHC
3       354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...
4                              2.0L I4 16V GDI DOHC Turbo
                              ...                        
4004                     6.0L W12 48V PDI DOHC Twin Turbo
4005        349.0HP 3.0L V6 Cylinder Engine Gasoline Fuel
4006                                             Electric
4007        450.0HP 3.5L V6 Cylinder Engine Gasoline Fuel
4008         248.0HP 2.0L 4 Cylinder Engine Gasoline Fuel
Name: engine, Length: 4009, dtype: object

In [36]:
### Tipos de Trnasmisión de motores

In [37]:
df['transmission'].unique()

array(['6-Speed A/T', '8-Speed Automatic', 'Automatic', '7-Speed A/T',
       'F', 'A/T', '8-Speed A/T', 'Transmission w/Dual Shift Mode',
       '9-Speed Automatic', '6-Speed M/T', '10-Speed A/T', '9-Speed A/T',
       '5-Speed A/T', '1-Speed A/T', 'Automatic CVT',
       '7-Speed Automatic with Auto-Shift', 'CVT-F', 'M/T',
       '6-Speed Automatic with Auto-Shift', '10-Speed Automatic',
       'CVT Transmission', '4-Speed A/T', '6-Speed Automatic',
       '4-Speed Automatic', '7-Speed M/T',
       '8-Speed Automatic with Auto-Shift', '5-Speed Automatic',
       '8-SPEED AT', '1-Speed Automatic', '5-Speed M/T', 'Manual',
       '6-Speed Manual', 'Variable', '2', '7-Speed Manual',
       'Automatic, 9-Spd 9G-Tronic', 'Automatic, 8-Spd',
       'Automatic, 8-Spd Sport w/Sport & Manual Modes', '–',
       'Auto, 6-Spd w/CmdShft', 'Transmission Overdrive Switch',
       '7-Speed Automatic', 'Automatic, 8-Spd PDK Dual-Clutch',
       'Automatic, 8-Spd M STEPTRONIC w/Drivelogic, Sport & Ma

In [38]:
df['speed'] = df['transmission'].str.extract(r'(\d+\.\d+|\d+)')

In [39]:
# Definir una función para extraer el texto deseado
def extraer_transmision(variable):
    variable = variable.lower()
    tipos_de_transmision = {'manual': ['manual', 'mt', 'm/t'],
                            'automatic': ['automatic', 'a/t', 'at'],
                            'dual': ['dual', 'at/mt']}
    
    for tipo, etiquetas in tipos_de_transmision.items():
        if any(etiqueta in variable for etiqueta in etiquetas):
            return tipo
    
    return 'other'


# Aplicar la función para extraer el texto deseado y crear una nueva columna
df['trans'] = df['transmission'].apply(extraer_transmision)

In [ ]:
df